## 爬取主题帖、回复贴信息

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

page_pa = 10#爬取目标页码数量设置    没有判断共有几页
Id = []
Title = []
Date = []
Tieba_name = []
Louzhu_text = []
Url = []
Page = []
Huitie_total = []
Page_total_in = []

Id_reply = []
Content = []
Reply_date = []
Father_id = []
Lou_num = []
Page_num = []
id = 0
id_reply = 0
for page in range(1,page_pa+1):
    url = "http://tieba.baidu.com/f/search/res?isnew=1&kw=&qw=%BA%BC%B5%E7%D4%F5%C3%B4%D1%F9&rn=10&un=&only_thread=1&sm=1&sd=&ed=&pn=" + str(page)
    #url = "http://tieba.baidu.com/f/search/res?isnew=1&kw=&qw=&rn=10&un=&only_thread=1&sm=1&sd=&ed=&pn=" + str(i)
    html = urlopen(url)
    soup = BeautifulSoup(html,"lxml")
    print ("正在爬取第%d页"%(page),end=' ')
    for i in range(len(soup.select('.p_title'))):
        id = id+1
        title = soup.select('.p_title')[i].get_text()
        date = soup.select('.p_green')[i].get_text()
        tieba_name = soup.select('.p_violet')[i].get_text()
        louzhu_text = soup.select('.p_content')[i].get_text()
        uuu = "http://tieba.baidu.com/" + soup.select('.p_title')[i].a.get("href")
        Id.append(id)
        Title.append(title)
        Date.append(date)
        Tieba_name.append(tieba_name)
        Louzhu_text.append(louzhu_text)#应该进入帖子爬取
        Url.append(uuu)
        Page.append(page)
        #进入帖子爬取 回帖数、页数信息
        html = urlopen(uuu)
        soup_linshi = BeautifulSoup(html,"lxml")
        #回帖数、页数信息
        huitie_total = soup_linshi.select('.l_reply_num')[0].findAll("span")[0].get_text()
        page_total_in = soup_linshi.select('.l_reply_num')[0].findAll("span")[1].get_text()
        Huitie_total.append(huitie_total)
        Page_total_in.append(page_total_in)
        print (".",end=' ')
        #爬取回复贴信息
        for i in range(len(soup_linshi.select('.d_post_content'))):#还没有考虑回复贴有多页
            id_reply = id_reply + 1
            father_id = id
            lou_num = i+1;
            page_num = 1;#还没有考虑多页回复
            #回复内容
            content = soup_linshi.select('.d_post_content')[i].get_text().lstrip()
            #回复时间
            try:
                begin = soup_linshi.select(".l_post")[i].get("data-field").find("date") + 7
                if begin-7 != -1:#回复时间信息格式不一样，要判断
                    end = begin + 16
                    reply_date = soup_linshi.select(".l_post")[i].get("data-field")[begin:end]
                    #print (soup_linshi.select(".l_post")[i].get("data-field")[begin:end])
                else:
                    reply_date = "null"
                    #加入NULL
            except:
                reply_date = "null"
            Id_reply.append(id_reply)
            Content.append(content)
            Reply_date.append(reply_date)
            Father_id.append(father_id)
            Lou_num.append(lou_num)
            Page_num.append(page_num)
    print ("\n")

print ("已爬取到%d条主题帖数据，%d条回复贴数据"%(len(Id),len(Id_reply)))
print ("爬取完成，共爬取到%d条主题帖数据，%d条回复贴数据"%(len(Id),len(Id_reply)))

b'<!DOCTYPE html><!--STATUS OK--><html><head><meta charset="GBK"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta name="baidu-site-verification" content="jpBCrwX689" /><link rel="search" type="application/opensearchdescription+xml" href="/tb/cms/content-search.xml" title="\xb0\xd9\xb6\xc8\xcc\xf9\xb0\xc9" /><title>\xb0\xd9\xb6\xc8\xcc\xf9\xb0\xc9_\xba\xbc\xb5\xe7\xd4\xf5\xc3\xb4\xd1\xf9</title><script type="text/javascript">void function(t,e,n,a,o,i,r){t.alogObjectName=o,t[o]=t[o]||function(){(t[o].q=t[o].q||[]).push(arguments)},t[o].l=t[o].l||+new Date,a="https:"===t.location.protocol?"https://fex.bdstatic.com"+a:"http://fex.bdstatic.com"+a;var c=!0;if(t.alogObjectConfig&&t.alogObjectConfig.sample){var s=Math.random();t.alogObjectConfig.rand=s,s>t.alogObjectConfig.sample&&(c=!1)}c&&(i=e.createElement(n),i.async=!0,i.src=a+"?v="+~(new Date/864e5)+~(new Date/864e5),r=e.getElementsByTagName(n)[0],r.parentNode.insertBefore(i,r))}(window,document,"script","/hunter/alog/a

KeyboardInterrupt: 

## 存入数据库
数据类型还要完善

In [27]:
import pymysql

# 打开数据库连接
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='', db='recommend', charset='utf8')
# 创建游标
cursor = db.cursor()
#清空数据库表
sql = "DELETE FROM testmodel_hui_fu_tie"
cursor.execute(sql)
#存储到数据库
t = 0
error_num = 0
print("正在存储回复帖信息",end=" ")
for i in range(len(Id_reply)):
    # SQL 插入语句
    sql = "INSERT INTO testmodel_hui_fu_tie(Id,Content,Reply_date,Father_id,Lou_num,Page_num) VALUES ('" + str(Id_reply[i]) \
        + "', '" + Content[i] \
        + "', '" + Reply_date[i] \
        + "', '" + str(Father_id[i]) \
        + "', '" + str(Lou_num[i]) \
        + "', '" + str(Page_num[i]) \
        +"')"
    t = t + 1
    try:
        #执行sql语句
        cursor.execute(sql)
        # 提交到数据库执行
        db.commit()
        if t > len(Id_reply)/10:
            t = 0
            print(".",end=" ")
    except Exception as e:
        error_num = error_num + 1
        db.rollback()
print("回复帖信息存储完成,%d条存储错误"%error_num)

t = 0
error_num = 0
print("正在存储主题帖信息",end=" ")
#清空数据库表
sql = "DELETE FROM testmodel_zhu_ti_tie"
cursor.execute(sql)
#存储到数据库
for i in range(len(Id)):
    # SQL 插入语句
    sql = "INSERT INTO testmodel_zhu_ti_tie(id,Title,Date,Tieba_name,Louzhu_text,Huitie_total,Page_num,Page_total_in,Url) VALUES ('" + str(Id[i]) \
        + "', '" + Title[i] \
        + "', '" + Date[i] \
        + "', '" + Tieba_name[i] \
        + "', '" + Louzhu_text[i] \
        + "', '" + str(Huitie_total[i]) \
        + "', '" + str(Page[i]) \
        + "', '" + str(Page_total_in[i]) \
        + "', '" + Url[i] \
        +"')"
    t = t + 1
    try:
        #执行sql语句
        cursor.execute(sql)
        # 提交到数据库执行
        db.commit()
        if t > len(Id)/10:
            t = 0
            print(".",end=" ")
    except Exception as e:
        error_num = error_num + 1
        #print(str(e))
        db.rollback()
db.close()
print("主题帖信息存储完成,%d条存储错误"%error_num)

正在存储回复帖信息 . . . . . . . . . 回复帖信息存储完成,8条存储错误
正在存储主题帖信息 . . . . . . . . . 主题帖信息存储完成,3条存储错误


## 读取数据库
可按时间排序、热度、是否有回帖、相关度排序、情感排序

In [8]:
import pymysql
# 打开数据库连接
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='', db='recommend', charset='utf8')
# 创建游标
cursor = db.cursor()
try:
    # 执行SQL，并返回收影响行数
    cursor.execute("select * from recommend.testmodel_hui_fu_tie")
    results = cursor.fetchall()
    print ("回复贴共%d条数据"%len(results))
    i = 0
    for row in results:
        i = i+1
except:
    print ("Error: unable to fetch data")
    
try:
    # 执行SQL，并返回收影响行数
    cursor.execute("select * from recommend.testmodel_zhu_ti_tie")
    results = cursor.fetchall()
    print ("主题贴共%d条数据"%len(results))
    i = 0
    for row in results:
        i = i+1
except:
    print ("Error: unable to fetch data")

回复贴共505条数据
主题贴共91条数据


## 初步处理数据
去除无用数据

In [58]:
import pymysql

# 打开数据库连接
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='', db='recommend', charset='utf8')
# 创建游标
cursor = conn.cursor()

# 执行SQL，并返回收影响行数
cursor.execute("select id,Content from recommend.testmodel_hui_fu_tie;")
results = cursor.fetchall()
id_delete = []
for row in results:
        text2 = row[1]
        if(text2 == ''):
            id_delete.append(row[0])
        if(text2.find("顶")>=0):
            #print(text2)
            #print(row[0])
            id_delete.append(row[0])
        if(text2.find("同求")>=0):
            id_delete.append(row[0])
        if(text2.find("如题")>=0):
            id_delete.append(row[0])
            print(text2)
        if(len(text2)>220):#过长文本
            id_delete.append(row[0])
            print(text2)
            #print(row)
print(id_delete)
#修改
for id in id_delete:
    sql = "delete from recommend.testmodel_hui_fu_tie where id = '" + str(id) +"'";
    effect_row  = cursor.execute(sql)
    print (effect_row)
#保存提交  
conn.commit()
# 关闭游标
cursor.close()
# 关闭连接
conn.close()

[]


## 文本分类
直接匹配

In [ ]:
import pymysql

# 打开数据库连接
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='', db='recommend', charset='utf8')
# 创建游标
cursor = conn.cursor()

# 执行SQL，并返回收影响行数
cursor.execute("select Id,Title from recommend.testmodel_zhu_ti_tie;")
results = cursor.fetchall()
id_jisuanji = []
id_kaoyan = []
id_jiuye = []
id_kuaiji = []
id_dianzi = []
for row in results:
    #直接匹配
    text1 = "计算机"
    text2 = row[1]
    if(text2.find(text1)>0):
        id_jisuanji.append(row[0])
    text1 = "考研"
    if(text2.find(text1)>0):
        id_kaoyan.append(row[0])
    text1 = "就业"
    if(text2.find(text1)>0):
        id_jiuye.append(row[0])
    text1 = "会计"
    if(text2.find(text1)>0):
        id_kuaiji.append(row[0])
    text1 = "电子"
    if(text2.find(text1)>0):
        id_dianzi.append(row[0])
        
print(id_jisuanji)
print(id_kaoyan)
print(id_jiuye)
print(id_kuaiji)
print(id_dianzi)
#修改
for id in id_jisuanji:
    sql = "update recommend.testmodel_zhu_ti_tie set type = '计算机' where id = " + str(id);
    effect_row  = cursor.execute(sql)
    print (effect_row)
for id in id_kaoyan:
    sql = "update recommend.testmodel_zhu_ti_tie set type = '考研' where id = " + str(id);
    effect_row  = cursor.execute(sql)
    print (effect_row)
for id in id_jiuye:
    sql = "update recommend.testmodel_zhu_ti_tie set type = '就业' where id = " + str(id);
    effect_row  = cursor.execute(sql)
    print (effect_row)
for id in id_kuaiji:
    sql = "update recommend.testmodel_zhu_ti_tie set type = '会计' where id = " + str(id);
    effect_row  = cursor.execute(sql)
    print (effect_row)
for id in id_dianzi:
    sql = "update recommend.testmodel_zhu_ti_tie set type = '电子' where id = " + str(id);
    effect_row  = cursor.execute(sql)
    print (effect_row)
#保存提交  
conn.commit()
# 关闭游标
cursor.close()
# 关闭连接
conn.close()

## 查看文本分类结果

In [1]:
import pymysql

# 打开数据库连接
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='', db='recommend', charset='utf8')
# 创建游标
cursor = conn.cursor()

# 执行SQL，并返回收影响行数
cursor.execute("select Id,Title,type from recommend.testmodel_zhu_ti_tie;")
results = cursor.fetchall()
id_update = []
for row in results:
    #直接匹配
    text1 = "计算机"
    text2 = row[1]
    if(text2.find(text1)>0):
        print(row)
    text1 = "考研"
    if(text2.find(text1)>0):
        print(row)
    text1 = "就业"
    if(text2.find(text1)>0):
        print(row)
    text1 = "会计"
    if(text2.find(text1)>0):
        print(row)
    text1 = "电子"
    if(text2.find(text1)>0):
        print(row)

# 关闭游标
cursor.close()
# 关闭连接
conn.close()

(4, '杭电的计算机专业在浙江省竞争力怎么样 高三求解', '计算机')
(8, '杭电金融研究生怎么样ya,19考研一枚,求解～', '考研')
(9, '杭电计算机研究生院开的中日合作怎么样?请利益相关者提出见解,', '计算机')
(11, '杭电计算机考研怎么样?被公司认可度怎么样,这个学校的报录比怎', '考研')
(11, '杭电计算机考研怎么样?被公司认可度怎么样,这个学校的报录比怎', '考研')
(12, '请问杭电的电子科学与技术研究生出来工作怎么样,好找吗,待遇如', '电子')
(13, '有没有小姐姐小哥哥来说下杭电计算机怎么样?', '计算机')
(17, '杭电电子与通信工程专硕好考吗?复试怎么样啊,会不会刷太多人,', '电子')
(23, '杭电应用统计学专业考研怎么样', '考研')
(24, '本人考研党,想往图像处理方向发展,请问杭电的图像处理怎么样,', '考研')
(25, '求教:想考研,杭电的管理学院怎么样?', '考研')
(27, '杭电 会计专硕招多少人呢?就业怎么样哦', '会计')
(27, '杭电 会计专硕招多少人呢?就业怎么样哦', '会计')
(30, '杭电的电子信息工程研究生怎么样啊,好考吗', '电子')
(31, '杭电的电子信息工程怎么样啊,好考吗', '电子')
(32, '求问,杭电应统专硕就业情况怎么样?', '就业')
(37, '杭电会计专业怎么样', '会计')
(39, '杭电的会计怎么样呢?', '会计')
(41, '【大一萌新】杭电的电子和数学怎么样啊', '电子')
(44, '杭电的会计学可以,但是它的经济学和语言怎么样啊?', '会计')
(45, '在杭电学习计算机类,比如软件工程,信息安全之类,怎么样???', '计算机')
(56, '杭电的计算机和电子怎么样求告知', '电子')
(56, '杭电的计算机和电子怎么样求告知', '电子')
(58, '求问杭电就业怎么样', '就业')
(62, '杭电的电子信息材料怎么样', '电子')
(64, '有杭电计算机成教进的吗,怎么样', '计算机')
(70, '杭电的计算机专业到底怎么样啊', '计算机')
(81, '杭电计算机和软件怎么样', '计算机')
(82, '有师哥师姐知道杭电的电子信息材料怎

## 情感分析
可创建分析数据结果表

In [53]:
import pymysql
from aip import AipNlp
import json
""" 你的 APPID AK SK """
APP_ID = '11076682'
API_KEY = 'ypkft3BxZ5QBhsl43oHBnV2r'
SECRET_KEY = 'qfo5BGorlQGyyqCU2irGuRfi4DW9GtU7'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

#try:
# 执行SQL，并返回收影响行数
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='', db='recommend', charset='utf8')
# 创建游标
cursor = db.cursor()
cursor.execute("select * from recommend.testmodel_hui_fu_tie")
results = cursor.fetchall()
i = 0
print("正在对回复贴内容进行情感分析",end=" ")
t = 0
for row in results:
    if i<len(results):#分析全部数据
        i = i+1
        text = row[1]
        if text!="":#判断非空
            """ 调用情感倾向分析 """
            #print(text)
            try:
                emotion_value = client.sentimentClassify(text)['items'][0]['positive_prob']
            except:
                print("情感分析错误")
            try:
                sql = "UPDATE testmodel_hui_fu_tie SET `Emotion_value`='" + str(emotion_value) + "' WHERE `Id`='" + str(row[0]) + "'"
                #执行sql语句
                cursor.execute(sql)
                # 提交到数据库执行
                t = t+1
                db.commit()
                if t > len(results)/10:
                    t = 0
                    print(".",end=" ")
            except Exception as e:
                error_num = error_num + 1
                db.rollback()
print("情感分析完成")

正在对回复贴内容进行情感分析 . . 情感分析错误
. . . . . . 情感分析错误
. 情感分析完成


## 查看情感分析结果
先进行相关性判断，处理无关信息

In [3]:
import pymysql
# 打开数据库连接
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='', db='recommend', charset='utf8')
# 创建游标
cursor = db.cursor()
#try:
# 执行SQL，并返回收影响行数
cursor.execute("select * from recommend.hui_fu_tie")
results = cursor.fetchall()
print ("积极：")
i = 0
for row in results:
    if i<len(results):#查看所有结果
        try:
            if float(row[6]) > 0.9:
                print(row[1])
        except:
            ;
    else:
        break
print ("消极：-------------")    
i = 0
for row in results:
    if i<50:
        try:
            if float(row[6]) < 0.1:
                print(row[1])
        except:
            ;
    else:
        break
#except:
#    print ("Error: unable to fetch data")

积极：
挺好的 电子是杭电的优秀专业
作为前口语部成员 可以跟你说 社团还是很不错的 能学到很多东西 每周都有固定时间大部分在晚上 有外国友人过来一起交流 在英语角 也有很多其他的活动 英语话剧啊 大一 什么的
就业很好，工资很高
杭电的会计专业还是很不错的
哈哈，好不好吃你都得吃。
算比较不错的
杭电会记很好啊
很好
学长学姐，能不能问一下杭电的电气工程及其自动化怎么样，出来在当地好找工作吗，或者帮我推荐一下杭电比较好的专业
很好啊
嗯！很好！像比其他学校真的同等价位好到爆了吧。四五人间，空调，热水器，独卫，阳台，宽带
本人是贵州考生，不出意外可以来贵校，想了解一下贵校在专业上的实力怎么样，师资力量怎么样，生活方面都不在乎，谢谢！
专业实力感觉不错，师资力量几乎都是本部调来的老师
专业肯定是理科方面的专业出色，金融财管会计类的也可以。师资力量不是我说，大学基本靠自觉……
主要是最近几年计算机很火加之杭电正好专业都集中在这里，自然就火了
主要这两年就业不景气 计算机应该算是出来工资最高的专业之一了 省内的话除了浙大 杭电的计算机确实比工大的强 感觉工大的领导不重视计算机专业 工大的领导比较重视 化学机械之类工大比较老牌的强势专业
赞一下
很不错的
很好的
我是杭电去年毕业的小姐姐 哈哈哈 想送弟弟去读杭电的继续教育学院 想问问你们觉得怎么样 对了  弟弟2000年 会不会还有点小有没有啥专业推荐咧
杭电的成绩怎么样？------耐克阿迪新百伦乔丹 等各大品牌运动鞋 运动套装批发零售，支持专柜验货 价格绝对找不到第二家。 
现免费招收全国代理 零库存 零风险 月赚万元不是梦，带你发财带你飞，0元成就发财致富梦，
想致富狂加微信或者QQ咨询！
QQ：2736803522
微信：dk556628 

金职院成教还不错，我学姐就是成教旅游学院毕业的，现在出国带团都很好的，想了解可以帮你问问
杭电宿舍条件还是非常好的，周边吃喝玩乐也很方便
条件不错哦
想考研究生到杭电听说研究生是双人套房？周围交通便利吗吃的玩儿的多吗hhhhhhh一直都很喜欢杭州啊离成都也不远hhh
跟马云有渊源，马云杭州师范毕业后就在杭州电子科技大学当英语老师。马云的英语非常棒，应该是破格录取为教师的。
中国教育在线《杭州市2015年度接收高校毕业生就业报告》昨出炉2016-05-27 编辑厉皆蒙699万、